In [2]:
from pokemontcgsdk import Card
from pokemontcgsdk import Set
from pokemontcgsdk import Type
from pokemontcgsdk import Supertype
from pokemontcgsdk import Subtype
from pokemontcgsdk import Rarity

from pokemontcgsdk import RestClient

import pickle
from tqdm import tqdm
import os

with open("options.txt", "r") as file:
    option_lines = file.readlines()
    
api_key = option_lines[0].split(" = ")[1]

RestClient.configure(api_key)

#### Set Caching

In [25]:
sets = Set.all()
all_sets_data = []

for card_set in sets:
    all_sets_data.append({"image_URL":card_set.images.logo, "name":card_set.name, "series": card_set.series, "release_date": card_set.releaseDate, "id": card_set.id}) 

all_sets_data = list(sorted(all_sets_data, key = lambda item: item["release_date"], reverse=True))
print(all_sets_data)

[{'image_URL': 'https://images.pokemontcg.io/sv1/logo.png', 'name': 'Scarlet & Violet', 'series': 'Scarlet & Violet', 'release_date': '2023/03/31', 'id': 'sv1'}, {'image_URL': 'https://images.pokemontcg.io/swsh12pt5gg/logo.png', 'name': 'Crown Zenith Galarian Gallery', 'series': 'Sword & Shield', 'release_date': '2023/01/20', 'id': 'swsh12pt5gg'}, {'image_URL': 'https://images.pokemontcg.io/swsh12pt5/logo.png', 'name': 'Crown Zenith', 'series': 'Sword & Shield', 'release_date': '2023/01/20', 'id': 'swsh12pt5'}, {'image_URL': 'https://images.pokemontcg.io/svp/logo.png', 'name': 'Scarlet & Violet Black Star Promos', 'series': 'Scarlet & Violet', 'release_date': '2023/01/01', 'id': 'svp'}, {'image_URL': 'https://images.pokemontcg.io/swsh12/logo.png', 'name': 'Silver Tempest', 'series': 'Sword & Shield', 'release_date': '2022/11/11', 'id': 'swsh12'}, {'image_URL': 'https://images.pokemontcg.io/swsh12tg/logo.png', 'name': 'Silver Tempest Trainer Gallery', 'series': 'Sword & Shield', 'releas

In [26]:
import pickle

with open("all_sets.pkl", "wb") as file:
    pickle.dump(all_sets_data, file)

#### Card Caching

In [49]:
sets = Set.all()
all_set_ids = {card_set.id for card_set in sets}
set_id = next(iter(all_set_ids))
cards = Card.where(q=f'set.id:{set_id}')

In [62]:
import copy

set_length = min([len(card.id) for card in cards]) - 1
print(set_length)

cards_sorted = []
for card in cards:
    new_card = copy.deepcopy(card)
    new_card.id = card.id[6:]
    while len(new_card.id) < 3:
        new_card.id = "0" + new_card.id
    cards_sorted.append(new_card)

cards_sorted = sorted(cards_sorted, key = lambda x: x.id)

print([card.id for card in cards_sorted][0:100])

6
['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100']


In [3]:
sets = Set.all()
all_set_ids = {card_set.id for card_set in sets}

for set_id in tqdm(all_set_ids):
    # Don't download if it exists
    if os.path.exists(f"card_db/card_data/{set_id}.pkl"):
        continue

    # if a set is empty this will cause an error
    try:
        # Get all cards in the set
        cards = Card.where(q=f'set.id:{set_id}')

        # Sort the cards by id, this requires appending 0s to cards with id's under tripple digits
        set_length = min([len(card.id) for card in cards]) - 1
        cards_sorted = []
        for card in cards:
            card.id = card.id[set_length:]
            while len(card.id) < 3:
                card.id = "0" + card.id
            cards_sorted.append(card)

        cards_sorted = sorted(cards_sorted, key = lambda x: x.id)

        with open(f"card_db/card_data/{set_id}.pkl", "wb") as file:
            pickle.dump(cards, file)

    except:
        print(f"Issues with set: {set_id}")

 57%|█████▋    | 86/152 [04:58<02:45,  2.50s/it]

Issues with set: svp


100%|██████████| 152/152 [09:23<00:00,  3.71s/it]


In [4]:
print(cards[:5])

[Card(abilities=[Ability(name='Darkness Shade', text='Once during your turn, when you put Darkrai from your hand onto your Bench, you may choose 1 of the Defending Pokémon. That Pokémon is now Asleep.', type='Poké-Power')], artist='Mitsuhiro Arita', ancientTrait=None, attacks=[Attack(name='Dark Slumber', cost=['Darkness'], convertedEnergyCost=1, damage='10', text="At the end of your opponent's next turn, the Defending Pokémon is now Asleep."), Attack(name='Dark Resolve', cost=['Darkness', 'Darkness', 'Colorless'], convertedEnergyCost=3, damage='40', text='If the Defending Pokémon is Asleep, remove 4 damage counters from Darkrai.')], cardmarket=Cardmarket(url='https://prices.pokemontcg.io/cardmarket/dp5-3', updatedAt='2023/05/24', prices=CardmarketPrices(averageSellPrice=12.85, lowPrice=0.5, trendPrice=11.61, germanProLow=0.0, suggestedPrice=0.0, reverseHoloSell=1.14, reverseHoloLow=2.49, reverseHoloTrend=2.3, lowPriceExPlus=2.5, avg1=5.99, avg7=11.83, avg30=5.79, reverseHoloAvg1=1.95, 

In [19]:
for i, card in enumerate(cards):
    if i > 10:
        continue
    print(card.images)
    print(card.name)
    print(card.set.name)
    print()
    print("")

CardImage(small='https://images.pokemontcg.io/mcd19/1.png', large='https://images.pokemontcg.io/mcd19/1_hires.png')
Caterpie
McDonald's Collection 2019


CardImage(small='https://images.pokemontcg.io/mcd19/2.png', large='https://images.pokemontcg.io/mcd19/2_hires.png')
Alolan Exeggutor
McDonald's Collection 2019


CardImage(small='https://images.pokemontcg.io/si1/1.png', large='https://images.pokemontcg.io/si1/1_hires.png')
Mew
Southern Islands


CardImage(small='https://images.pokemontcg.io/mcd19/3.png', large='https://images.pokemontcg.io/mcd19/3_hires.png')
Magmar
McDonald's Collection 2019


CardImage(small='https://images.pokemontcg.io/ru1/3.png', large='https://images.pokemontcg.io/ru1/3_hires.png')
Ninetales
Pokémon Rumble


CardImage(small='https://images.pokemontcg.io/ru1/1.png', large='https://images.pokemontcg.io/ru1/1_hires.png')
Venusaur
Pokémon Rumble


CardImage(small='https://images.pokemontcg.io/ru1/2.png', large='https://images.pokemontcg.io/ru1/2_hires.png')
Cherrim
